# Day096
## Keras 中的 CNN layers
影像經過卷積後稱作特徵圖 (feature map)，經過多次卷積層後，特徵圖的尺寸 (width, height) 會越來越小，但是通道數 (Channel) 則會越來越大。

### Conv2D 中的參數意義:
- filters：濾波器的數量。此數字會等於做完卷積後特徵圖的通道數，通常設定為 2 的 n 次方
- kernel_size：濾波器的大小。通常都是使用 3x3 或是 5x5
- input_shape：只有對影像做第一次卷積時要指定，之後 Keras 會自動計算input_shape
- strides：做卷積時，濾波器移動的步長。此處的 stirides 就是 1 (一次移動一格)
- padding：是否要對輸入影像的邊緣補值。此處的 padding=same (邊緣補一層 0)，稱為 same 的原因是因為做完 padding 再卷積後，輸出的特徵圖尺寸與輸入影像的尺寸不會改變

### SeparableConv2D 
- 全名稱做 Depthwise Separable Convolution，與常用的 Conv2D 效果類似，但是參數量可以大幅減少，減輕對硬體的需求
- 對影像做兩次卷積，第一次稱為 DetphWise Conv，對影像的三個通道獨立做卷積，得到三張特徵圖；第二次稱為 PointWise Conv，使用 1x1 的 filter 尺寸做卷積。兩次卷積結合起來可以跟常用的卷積達到接近的效果，但參數量卻遠少於常見的卷積，更多資訊可參考[連結](http://www.icode9.com/content-4-93052.html)

####  SeparableConv2D 中的參數意義
- filters, kernel_size, strides, padding 都與 Conv2D 相同
- depth_multiplier:　在做 DepthWise Conv 時，輸出的特徵圖 Channel 數量會是 filters * depth_multiplier，預設為 1

In [1]:
from keras.layers import Conv2D, SeparableConv2D, Input
from keras.models import Model

Using TensorFlow backend.


In [2]:
input_image = Input((224, 224, 3))
feature_maps = Conv2D(filters=32, kernel_size=(3,3))(input_image)
feature_maps2 = Conv2D(filters=64, kernel_size=(3,3))(feature_maps)
model = Model(inputs=input_image, outputs=feature_maps2)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 64)      18496     
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
_________________________________________________________________


## 可以看到經過兩次 Conv2D，如果沒有設定 padding="SAME"，圖就會越來越小，同時特徵圖的 channel 數與 filters 的數量一致

In [4]:
input_image = Input((224, 224, 3))
feature_maps = SeparableConv2D(filters=32, kernel_size=(3,3))(input_image)
feature_maps2 = SeparableConv2D(filters=64, kernel_size=(3,3))(feature_maps)
model = Model(inputs=input_image, outputs=feature_maps2)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 222, 222, 32)      155       
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 220, 220, 64)      2400      
Total params: 2,555
Trainable params: 2,555
Non-trainable params: 0
_________________________________________________________________


## 可以看到使用 Seperable Conv2D，即使模型設置都一模一樣，但是參數量明顯減少非常多！

## 作業

請閱讀 Keras 官方範例 [mnist_cnn.py](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)  

並回答下列問題。僅有 70 行程式碼，請確保每一行的程式碼你都能夠理解目的

1. 是否有對資料做標準化 (normalization)? 如果有，在哪幾行?
> 有，在37、38行
2. 使用的優化器 Optimizer 為何?
> Adadelta
3. 模型總共疊了幾層卷積層?
> 2層
4. 模型的參數量是多少?
> 1,199,882

### 程式碼如下

In [6]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 11s 189us/step - loss: 0.2585 - acc: 0.9214 - val_loss: 0.0567 - val_acc: 0.9807
Epoch 2/12
60000/60000 [==============================] - 6s 105us/step - loss: 0.0888 - acc: 0.9740 - val_loss: 0.0455 - val_acc: 0.9852
Epoch 3/12
60000/60000 [==============================] - 6s 105us/step - loss: 0.0648 - acc: 0.9802 - val_loss: 0.0357 - val_acc: 0.9882
Epoch 4/12
60000/60000 [==============================] - 6s 106us/step - loss: 0.0543 - acc: 0.9838 - val_loss: 0.0361 - val_acc: 0.9875
Epoch 5/12
60000/60000 [==============================] - 6s 105us/step - loss: 0.0467 - acc: 0.9861 - val_loss: 0.0306 - val_acc: 0.9903
Epoch 6/12
60000/60000 

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________